In [ ]:
import numpy               as np
from geopandas             import GeoDataFrame
from glass.rd          import tbl_to_obj
from glass.g.mng.prj      import project
from glass.wt              import obj_to_tbl
from glass.g.dp.cg.obj import pntDf_to_convex_hull
from glass.g.to.geom      import pnt_dfwxy_to_geodf
from glass.wt.shp       import df_to_shp
from glass.adv.floc.pntint import df_direct_intersection, df_lateral_intersection

In [ ]:
contributions = r'F:\fireloc\tst_miranda\pnt_obs_sample_date.shp'

#declinacao = -(7 + (36/60.0))
declinacao = 0

col_direction = 'direct2'

pnt_result  = r'F:\fireloc\tst_miranda\fire_prediction3.shp'
poly_result = r'F:\fireloc\tst_miranda\fire_polygon3.shp'

In [ ]:
pnts = tbl_to_obj(contributions)

pnts = pnts[[col_direction, 'geometry']]

pnts = project(pnts, None, 20790, inEPSG=3763, gisApi='pandas')

pnts['x_coord'] = pnts.geometry.x
pnts['y_coord'] = pnts.geometry.y
pnts['relatio'] = 1
pnts['fid']     = pnts.index

pnts = pnts.merge(pnts, how='outer', left_on='relatio', right_on='relatio')
pnts = pnts[pnts.fid_x < pnts.fid_y]

pnts.drop(['geometry_x', 'geometry_y', 'relatio'], axis=1, inplace=True)

pnts.rename(columns={
    col_direction + '_x' : 'direcao_a_q', col_direction + '_y' : 'direcao_b_q',
    'x_coord_x' : 'a_x', 'y_coord_x' : 'a_y', 'x_coord_y' : 'b_x', 'y_coord_y' : 'b_y'
}, inplace=True)

pnts['direcao_a_q'] = pnts['direcao_a_q'] + declinacao
pnts['direcao_b_q'] = pnts['direcao_b_q'] + declinacao

In [ ]:
pnts.head(100)

In [ ]:
pnts = df_direct_intersection(
    pnts, 'q', 'a_x', 'a_y', 'b_x', 'b_y',
    'direcao_a_q', 'direcao_b_q'
)

pnts = df_lateral_intersection(
    pnts, 'l_q', 'a_x', 'a_y', 'b_x', 'b_y',
    'direcao_a_q', 'direcao_b_q'
)

In [ ]:
pnts['q_P'] = np.where((pnts.q_P == np.inf) | (pnts.q_P == -np.inf), pnts.l_q_P, pnts.q_P)
pnts['q_M'] = np.where((pnts.q_M == np.inf) | (pnts.q_M == -np.inf), pnts.l_q_M, pnts.q_M)
pnts.drop(['l_q_P', 'l_q_M'], axis=1, inplace=True)

In [ ]:
print(pnts.shape)
pnts = pnts[~pnts.isin([np.nan, np.inf, -np.inf]).any(1)]
print(pnts.shape)

In [ ]:
hull = ConvexHull(pnts[['q_M', 'q_P']])

poly = geometry.Polygon([[
    pnts.q_M.iloc[idx], pnts.q_P.iloc[idx]
] for idx in hull.vertices])

convex_df = GeoDataFrame(
    [1], columns=['cat'],
    crs='EPSG:20790',
    geometry=[poly]
)

In [ ]:
if not poly_result:
    import os
    from glass.pys.oss import fprop
    
    poly_result = os.path.join(os.path.dirname(pnt_result), fprop(pnt_result, 'fn') + '_convex.shp')

poly_result = pntDf_to_convex_hull(pnts, 'q_M', 'q_P', 20790, 3763)

In [ ]:
convex_df.head(5)

In [ ]:
pnts = pnt_dfwxy_to_geodf(pnts, 'q_M', 'q_P', 20790)
pnts = project(pnts, None, 3763, inEPSG=20790, gisApi='pandas')

convex_df = project(convex_df, None, 3763, inEPSG=20790, gisApi='pandas')

df_to_shp(pnts, pnt_result)
if not poly_result:
    import os
    from glass.pys.oss import fprop
    
    poly_result = os.path.join(os.path.dirname(pnt_result), fprop(pnt_result, 'fn') + '_convex.shp')

df_to_shp(convex_df, poly_result)